In [ ]:
# use both the path cost (g) and heuristic (h) to calculate total cost f = g + h.

class Location:
    def __init__(self, name, distance_to_destination):
        self.name = name
        self.distance = distance_to_destination  # Straight line distance to destination
        self.neighbors = {}  # Dictionary of neighbor locations and road distances
        self.traffic_level = 1  # 1: Clear, 2: Moderate, 3: Heavy
    
    def add_neighbor(self, neighbor, road_distance):
        self.neighbors[neighbor] = road_distance
        # Make connection bidirectional
        neighbor.neighbors[self] = road_distance

def calculate_travel_time(distance, traffic_level):
    # Assume average speed: 60km/h with clear traffic
    # Traffic levels affect speed: Clear = 1x, Moderate = 1.5x, Heavy = 2x
    base_time = distance / 60  # Time in hours
    return base_time * traffic_level

def a_star_navigation(start, destination):
    open_set = {start}  # Locations to be evaluated
    closed_set = set()  # Already evaluated locations
    
    # Store g_scores (time from start) and f_scores (estimated total time)
    g_score = {start: 0}
    f_score = {start: start.distance}
    
    # Store the path
    came_from = {}
    
    while open_set:
        # Find location with lowest f_score
        current = min(open_set, key=lambda loc: f_score.get(loc, float('inf')))
        
        if current == destination:
            return reconstruct_path(came_from, current)
        
        open_set.remove(current)
        closed_set.add(current)
        
        # Check all neighbors
        for neighbor, road_distance in current.neighbors.items():
            if neighbor in closed_set:
                continue
            
            # Calculate time including traffic
            travel_time = calculate_travel_time(road_distance, neighbor.traffic_level)
            tentative_g_score = g_score[current] + travel_time
            
            if neighbor not in open_set:
                open_set.add(neighbor)
            elif tentative_g_score >= g_score.get(neighbor, float('inf')):
                continue
            
            # This path is the best so far
            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = g_score[neighbor] + neighbor.distance

    return None

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

# Example usage
def main():
    # Create locations with distances to final destination (in km)
    home = Location("Home", 15)
    market = Location("Market", 12)
    school = Location("School", 8)
    park = Location("Park", 5)
    mall = Location("Mall", 3)
    office = Location("Office", 0)  # Destination
    
    # Add road connections with distances (in km)
    home.add_neighbor(market, 5)
    home.add_neighbor(school, 7)
    market.add_neighbor(park, 6)
    school.add_neighbor(park, 4)
    park.add_neighbor(mall, 3)
    mall.add_neighbor(office, 3)
    
    # Add some traffic conditions
    market.traffic_level = 2  # Moderate traffic
    park.traffic_level = 3    # Heavy traffic
    
    # Find best route
    route = a_star_navigation(home, office)
    
    # Print the route
    print("Best route to Office:")
    if route:
        total_distance = 0
        total_time = 0
        
        for i in range(len(route)-1):
            current = route[i]
            next_loc = route[i+1]
            distance = current.neighbors[next_loc]
            time = calculate_travel_time(distance, next_loc.traffic_level)
            
            total_distance += distance
            total_time += time
            
            print(f"{current.name} -> {next_loc.name}")
            print(f"  Distance: {distance}km")
            print(f"  Traffic Level: {'Clear' if next_loc.traffic_level == 1 else 'Moderate' if next_loc.traffic_level == 2 else 'Heavy'}")
            print(f"  Estimated Time: {time:.1f} hours")
            print()
        
        print(f"Total Distance: {total_distance}km")
        print(f"Total Estimated Time: {total_time:.1f} hours")
    else:
        print("No route found!")

if __name__ == "__main__":
    main()

Best route to Office:
Home -> School
  Distance: 7km
  Traffic Level: Clear
  Estimated Time: 0.1 hours

School -> Park
  Distance: 4km
  Traffic Level: Heavy
  Estimated Time: 0.2 hours

Park -> Mall
  Distance: 3km
  Traffic Level: Clear
  Estimated Time: 0.1 hours

Mall -> Office
  Distance: 3km
  Traffic Level: Clear
  Estimated Time: 0.1 hours

Total Distance: 17km
Total Estimated Time: 0.4 hours
